In [246]:
import glob
import pandas as pd
import numpy as np
import os
from IPython.display import HTML

In [247]:
!ls ../../ppc/scripts

autotune.sh
autotune_12000.sh
autotune_zi.sh
autotune_zi_12000.sh
brain_small_nb_1200_50_results.json
brain_small_zinb_1200_50_results.json
corr_nb_dataset_2000_log_normal
cortex
cortex_nb_1200_50_results.json
cortex_nb_20_2_results.json
cortex_zinb_1200_50_results.json
cortex_zinb_20_2_results.json
data
hemato_nb_1200_50_results.json
hemato_zinb_1200_50_results.json
log_poisson_nb_dataset_10000
log_poisson_nb_dataset_12000
log_poisson_nb_dataset_6000
log_poisson_nb_dataset_6000_nb_1200_1_results.json
log_poisson_nb_dataset_6000_zinb_1200_1_results.json
log_poisson_nb_dataset_8000
log_poisson_zinb_dataset_1000
log_poisson_zinb_dataset_12000
log_poisson_zinb_dataset_6000
log_poisson_zinb_dataset_8000
log_poisson_zinb_dataset_8000_old
model_eval.sh
model_eval_12000.sh
model_eval_zi.sh
model_eval_zi_12000.sh
news
pbmc_nb_1200_50_results.json
pbmc_zinb_1200_50_results.json
real_datasets_autotune_eval.sh
real_datasets_autotune_eval_0522_log.txt
real_datasets_autotune_eval_catchup.sh
result.

In [249]:
data_path = '../../ppc/scripts/'
dataset_name = 'log_poisson_nb_dataset_12000'
data_files = sorted(glob.glob(os.path.join(data_path, '{}/*.p'.format(dataset_name))))
for data_file in data_files:
    print(data_file)
data_names = ['nb', 'zifa_full', 'zinb']

In [238]:
dfs = []
for data_name, f in zip(data_names, data_files):
    if data_name != 'zifa_full':
        
        #my_df = pd.read_csv(f, sep='\t')
        my_df = pd.read_pickle(f)
        my_df.loc[:, 'model'] = data_name
        dfs.append(my_df)
        print(data_name, f)
        display(my_df)
df = pd.concat(dfs, axis=0)

nb ../../ppc/scripts/log_poisson_nb_dataset_8000\nb_log_poisson_nb_dataset_8000.p


,gene_ll_gene_ll,imputation_median_imputation_score,ll_ll,t_cv_ks_pval,t_cv_ks_stat,t_cv_phi_gen_gene,t_cv_phi_real_gene,t_dropout_ks_pval,t_dropout_ks_stat,t_dropout_phi_gen_gene,t_dropout_phi_real_gene,t_ratio_ks_pval,t_ratio_ks_stat,t_ratio_phi_gen_gene,t_ratio_phi_real_gene,model
0,"[3.6479888, 3.6044796, 1.3518083, 1.8827064, 3...",4.483450,170.912468,7.159507e-05,0.44,"[[1.2215785, 1.2676736, 1.2497014, 1.2074792, ...","[1.2827842, 1.2813728, 1.5987053, 1.4328156, 1...",1.714441e-02,0.30,"[[0.00453125, 0.0065625, 0.00875, 0.0078125, 0...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.002112,0.36,"[[1.9983425906674193, 1.611843928556687, 1.853...","[0.7816384019907211, 0.5294210390925159, 1394....",nb
1,"[3.4504526, 3.3738508, 1.4110588, 1.8302803, 3...",4.735722,168.880673,7.226654e-16,0.82,"[[1.416455, 1.4339368, 1.4996324, 1.5171465, 1...","[1.282783, 1.2813752, 1.5987061, 1.4328171, 1....",3.584201e-01,0.18,"[[0.00625, 0.00609375, 0.00609375, 0.00671875,...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.995845,0.08,"[[0.8013295042390819, 0.9053889314048601, 0.90...","[0.7816384019907211, 0.5294210390925159, 1394....",nb
2,"[3.4642215, 3.4259024, 1.4642122, 1.8389534, 3...",5.054935,170.975090,1.256050e-07,0.56,"[[1.2956841, 1.3055806, 1.2660362, 1.287113, 1...","[1.2827832, 1.2813748, 1.5987157, 1.4328156, 1...",2.407920e-01,0.20,"[[0.005, 0.005625, 0.0065625, 0.00640625, 0.00...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.677937,0.14,"[[1.5358874382334058, 1.051114179929621, 1.666...","[0.7816384019907211, 0.5294210390925159, 1394....",nb
3,"[3.4280868, 3.3468468, 1.3506737, 1.8278832, 3...",4.170612,168.151075,1.086624e-13,0.76,"[[1.4092052, 1.4484106, 1.4167134, 1.4272952, ...","[1.2827827, 1.2813736, 1.5987083, 1.4328166, 1...",2.111951e-03,0.36,"[[0.00578125, 0.00625, 0.00734375, 0.008125, 0...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.008899,0.32,"[[1.547552980637292, 1.2791293433233517, 1.045...","[0.7816384019907211, 0.5294210390925159, 1394....",nb
4,"[3.4792604, 3.3828142, 1.4057317, 1.8307011, 3...",4.841706,168.915026,1.256050e-07,0.56,"[[1.371093, 1.3691097, 1.3084354, 1.3696098, 1...","[1.282784, 1.2813747, 1.5987042, 1.4328182, 1....",1.714441e-02,0.30,"[[0.008125, 0.00640625, 0.005625, 0.00515625, ...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.004428,0.34,"[[1.160111233199152, 1.0944535311337757, 1.581...","[0.7816384019907211, 0.5294210390925159, 1394....",nb
5,"[3.488853, 3.4317849, 1.3887478, 1.822879, 3.3...",4.543348,169.640986,1.248758e-16,0.84,"[[1.6254227, 1.515882, 1.5128666, 1.5025725, 1...","[1.282783, 1.281374, 1.5987025, 1.4328182, 1.2...",1.714441e-02,0.30,"[[0.00625, 0.00578125, 0.00578125, 0.00625, 0....","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.840948,0.12,"[[0.9009751402785596, 1.0775023997344053, 1.05...","[0.7816384019907211, 0.5294210390925159, 1394....",nb
6,"[3.6233041, 3.510265, 1.4437035, 1.8669746, 3....",4.450342,171.894278,2.068458e-17,0.86,"[[1.5615594, 1.542089, 1.5488564, 1.516299, 1....","[1.2827837, 1.2813731, 1.5987097, 1.4328134, 1...",2.763769e-05,0.46,"[[0.0125, 0.01421875, 0.01359375, 0.01265625, ...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.000966,0.38,"[[2.661489608259214, 2.3649031648559284, 2.604...","[0.7816384019907211, 0.5294210390925159, 1394....",nb
7,"[3.4994879, 3.4458916, 1.3765733, 1.8349639, 3...",4.205555,169.724550,2.131758e-14,0.78,"[[1.4833829, 1.4587702, 1.432036, 1.5512737, 1...","[1.2827836, 1.2813739, 1.5987084, 1.4328153, 1...",7.159507e-05,0.44,"[[0.0128125, 0.01109375, 0.013125, 0.0090625, ...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.000966,0.38,"[[2.154244069873157, 2.0614833744680023, 1.669...","[0.7816384019907211, 0.5294210390925159, 1394....",nb
8,"[4.081543, 4.0761123, 1.5122877, 1.9932278, 3....",5.795495,183.853618,4.998802e-19,0.90,"[[1.6054945, 1.640752, 1.6680505, 1.6460373, 1...","[1.2827836, 1.2813736, 1.5987053, 1.4328178, 1...",1.71444

zinb ../../ppc/scripts/log_poisson_nb_dataset_8000\zinb_log_poisson_nb_dataset_8000.p


,gene_dropout_gene_dropout_logits_all,gene_dropout_gene_dropout_logits_test,gene_dropout_gene_dropout_logits_train,gene_dropout_gene_dropout_probs_all,gene_dropout_gene_dropout_probs_test,gene_dropout_gene_dropout_probs_train,gene_ll_gene_ll,imputation_median_imputation_score,ll_ll,t_cv_ks_pval,...,t_dropout_phi_gen_gene,t_dropout_phi_real_gene,t_ratio_ks_pval,t_ratio_ks_stat,t_ratio_phi_gen_gene,t_ratio_phi_real_gene,zero_params_zero_dropout_probs_all,zero_params_zero_rates_all,zero_params_zero_scales_all,model
0,"[-6.4891915, -6.6107287, -2.4511063, -3.765836...","[-6.4989233, -6.611767, -2.4571671, -3.7822442...","[-6.486762, -6.6104655, -2.4495914, -3.7617342...","[0.0016817132, 0.0014302172, 0.12253273, 0.046...","[0.0016584996, 0.0014253506, 0.121127196, 0.04...","[0.001687516, 0.0014314328, 0.12288411, 0.0473...","[4.1143217, 4.3431134, 1.5409235, 1.9062115, 4...",5.971622,186.522908,4.998802e-19,...,"[[0.00546875, 0.006875, 0.006875, 0.0059375, 0...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.154629,0.22,"[[0.586207532213763, 0.6012295828596333, 0.656...","[0.7816384019907211, 0.5294210390925159, 1394....","[0.021392697, 0.01825057, 0.0018141789, 0.0120...","[0.9102469, 1.8763392, 10.67333, 2.1789773, 6....","[0.001385834, 0.0028566914, 0.016249945, 0.003...",zinb
1,"[-6.4097958, -6.54981, -2.4831388, -3.4823968,...","[-6.4176335, -6.55894, -2.5063233, -3.483441, ...","[-6.4078345, -6.547533, -2.4773421, -3.4821396...","[0.0019463669, 0.0016920858, 0.11631266, 0.052...","[0.0019382049, 0.001687212, 0.1128602, 0.05131...","[0.001948407, 0.0016933031, 0.11717591, 0.0522...","[3.563652, 3.68301, 1.5343579, 1.8654552, 3.45...",4.740133,172.800045,1.086624e-13,...,"[[0.00703125, 0.005625, 0.0059375, 0.0059375, ...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.507658,0.16,"[[0.95035001619292, 0.644263893402665, 0.82241...","[0.7816384019907211, 0.5294210390925159, 1394....","[0.08376062, 0.008043575, 0.0014349519, 0.0102...","[4.5905657, 6.795329, 26.903147, 5.8704696, 7....","[0.0026189787, 0.0038768253, 0.015348602, 0.00...",zinb
2,"[-6.9322295, -7.088309, -2.4849832, -3.8177607...","[-6.936764, -7.091029, -2.524563, -3.8086045, ...","[-6.931091, -7.08763, -2.4750934, -3.8200412, ...","[0.001096464, 0.000893394, 0.12908787, 0.05289...","[0.0010856793, 0.000887983, 0.123933576, 0.052...","[0.0010991604, 0.00089474604, 0.13037631, 0.05...","[3.935577, 3.814521, 1.3714645, 1.9144558, 3.7...",5.373360,177.623699,4.020230e-07,...,"[[0.01140625, 0.0109375, 0.01296875, 0.0134375...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.000001,0.52,"[[3.7701784225008046, 2.9333298026572754, 3.34...","[0.7816384019907211, 0.5294210390925159, 1394....","[0.110022195, 0.0034447252, 0.002830809, 0.002...","[1.4673561, 5.544993, 8.899786, 4.651756, 0.15...","[0.002240676, 0.0084672915, 0.013590114, 0.007...",zinb
3,"[-6.8156176, -6.9300094, -2.481998, -3.8837304...","[-6.813526, -6.926991, -2.4760566, -3.8893933,...","[-6.816138, -6.930751, -2.4834793, -3.8823195,...","[0.0012048688, 0.0010482629, 0.12599993, 0.049...","[0.0012046554, 0.0010488648, 0.1255678, 0.0483...","[0.0012049204, 0.0010481123, 0.12610802, 0.049...","[3.5091405, 3.472914, 1.3866929, 1.8250768, 3....",4.542500,170.790355,1.248758e-16,...,"[[0.00828125, 0.00625, 0.006875, 0.0071875, 0....","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.017144,0.30,"[[1.2201658381508742, 1.1940525762931078, 1.07...","[0.7816384019907211, 0.5294210390925159, 1394....","[0.0035778128, 0.08544791, 0.0008726382, 0.080...","[5.973075, 1.2082229, 91.575455, 1.221372, 3.6...","[0.001234341, 0.00024968028, 0.018924145, 0.00...",zinb
4,"[-6.533313, -6.704564, -3.2697275, -3.3706708,...","[-6.5311937, -6.70035, -3.2895849, -3.3592803,...","[-6.5338306, -6.7056203, -3.264758, -3.373513,...","[0.0015849815, 0.0013178206, 0.061430383, 0.06...","[0.0015873014, 0.0013223207, 0.059488073, 0.06...","[0.0015844025, 0.0013166995, 0.061915975, 0.06...","[3.6442833, 3.5430553, 

C:\Users\ocliv\Anaconda3\envs\scvienv\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [239]:
df

,gene_dropout_gene_dropout_logits_all,gene_dropout_gene_dropout_logits_test,gene_dropout_gene_dropout_logits_train,gene_dropout_gene_dropout_probs_all,gene_dropout_gene_dropout_probs_test,gene_dropout_gene_dropout_probs_train,gene_ll_gene_ll,imputation_median_imputation_score,ll_ll,model,...,t_dropout_ks_stat,t_dropout_phi_gen_gene,t_dropout_phi_real_gene,t_ratio_ks_pval,t_ratio_ks_stat,t_ratio_phi_gen_gene,t_ratio_phi_real_gene,zero_params_zero_dropout_probs_all,zero_params_zero_rates_all,zero_params_zero_scales_all
0,NaN,NaN,NaN,NaN,NaN,NaN,"[3.6479888, 3.6044796, 1.3518083, 1.8827064, 3...",4.483450,170.912468,nb,...,0.30,"[[0.00453125, 0.0065625, 0.00875, 0.0078125, 0...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.002112,0.36,"[[1.9983425906674193, 1.611843928556687, 1.853...","[0.7816384019907211, 0.5294210390925159, 1394....",NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,"[3.4504526, 3.3738508, 1.4110588, 1.8302803, 3...",4.735722,168.880673,nb,...,0.18,"[[0.00625, 0.00609375, 0.00609375, 0.00671875,...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.995845,0.08,"[[0.8013295042390819, 0.9053889314048601, 0.90...","[0.7816384019907211, 0.5294210390925159, 1394....",NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,"[3.4642215, 3.4259024, 1.4642122, 1.8389534, 3...",5.054935,170.975090,nb,...,0.20,"[[0.005, 0.005625, 0.0065625, 0.00640625, 0.00...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.677937,0.14,"[[1.5358874382334058, 1.051114179929621, 1.666...","[0.7816384019907211, 0.5294210390925159, 1394....",NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,"[3.4280868, 3.3468468, 1.3506737, 1.8278832, 3...",4.170612,168.151075,nb,...,0.36,"[[0.00578125, 0.00625, 0.00734375, 0.008125, 0...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.008899,0.32,"[[1.547552980637292, 1.2791293433233517, 1.045...","[0.7816384019907211, 0.5294210390925159, 1394....",NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,"[3.4792604, 3.3828142, 1.4057317, 1.8307011, 3...",4.841706,168.915026,nb,...,0.30,"[[0.008125, 0.00640625, 0.005625, 0.00515625, ...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.004428,0.34,"[[1.160111233199152, 1.0944535311337757, 1.581...","[0.7816384019907211, 0.5294210390925159, 1394....",NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,"[3.488853, 3.4317849, 1.3887478, 1.822879, 3.3...",4.543348,169.640986,nb,...,0.30,"[[0.00625, 0.00578125, 0.00578125, 0.00625, 0....","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.840948,0.12,"[[0.9009751402785596, 1.0775023997344053, 1.05...","[0.7816384019907211, 0.5294210390925159, 1394....",NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,"[3.6233041, 3.510265, 1.4437035, 1.8669746, 3....",4.450342,171.894278,nb,...,0.46,"[[0.0125, 0.01421875, 0.01359375, 0.01265625, ...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.000966,0.38,"[[2.661489608259214, 2.3649031648559284, 2.604...","[0.7816384019907211, 0.5294210390925159, 1394....",NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,"[3.4994879, 3.4458916, 1.3765733, 1.8349639, 3...",4.205555,169.724550,nb,...,0.44,"[[0.0128125, 0.01109375, 0.013125, 0.0090625, ...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.000966,0.38,"[[2.154244069873157, 2.0614833744680023, 1.669...","[0.7816384019907211, 0.5294210390925159, 1394....",NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,"[4.081543, 4.0761123, 1.5122877, 1.9932278, 3....",5.795495,183.853618,nb,...,0.30,"[[0.003125, 0.00390625, 0.0028125, 0.00171875,...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.000072,0.44,"[[0.3630962267642997, 0.15235797208916396, 0.4...","[0.7816384019907211, 0.5294210390925159, 1394....",NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,"[3.542022, 3.494728, 1.3690057, 1.8306767, 3.4...",5.280876,169.714959,nb,...,0.26,"[[0.00625, 0.00640625, 0.00609375, 0.0053125, ...","[0.004375, 0.003125, 0.4790625, 0.3228125, 0.0...",0.008899,0.32,"[[1.701303316240064, 1.1909147931728896, 0.971...","[0.7816384019907211, 0.5294210390925159, 1394....",NaN,NaN,NaN


In [240]:
def str_to_array(string):
    list_nbs = string[1:-1].replace('\n','').replace('  ',' ').split(' ')
    ind = 0
    
    while ind < len(list_nbs):
        if list_nbs[ind] == '':
            del list_nbs[ind]
        else:
            ind += 1
    
    print(len(list_nbs))
            
    for ind in range(len(list_nbs)):
        list_nbs[ind] = float(list_nbs[ind])
    return np.array(list_nbs)

gene_dropout_all = df[df.model == 'zinb'].zero_params_zero_scales_all
gene_dropout_all_cont = np.vstack(list(gene_dropout_all))
print(gene_dropout_all_cont, gene_dropout_all_cont.shape)
print(dataset_name + "_2 : ",  gene_dropout_all_cont.mean())

In [241]:
metrics = ['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat', 't_ratio_ks_stat', 't_cv_ks_stat']
h1_hypothesis = ['larger', 'larger', 'larger', 'larger', 'larger']
h1_hypothesis_bis = ['smaller', 'smaller', 'smaller', 'smaller', 'smaller']

df_nb = df.loc[df.model=='nb', metrics]
df_zinb = df.loc[df.model=='zinb', metrics]



from statsmodels.stats.weightstats import ttest_ind

def get_pvals(gby, other_df):
    my_df = gby[metrics]
    print(gby)
    assert my_df.shape[1] == len(metrics)
    pvals = []
    for idx, alternative in enumerate(h1_hypothesis):
        assert len(other_df.values[:, idx]) != len(h1_hypothesis), (len(other_df.values[:, idx]), len(h1_hypothesis))
        _, pval, _ = ttest_ind(other_df.values[:, idx], my_df.values[:, idx], alternative=alternative)
        pvals.append(pval)
    return np.array(pvals)

pvals_against_zinb = df.groupby('model').apply(get_pvals, other_df=df_zinb)
pvals_against_zinb = (pvals_against_zinb
         .apply(lambda x: pd.Series(x))
         .T)
pvals_against_zinb = pvals_against_zinb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

pvals_against_nb = df.groupby('model').apply(get_pvals, other_df=df_nb)
pvals_against_nb = (pvals_against_nb
         .apply(lambda x: pd.Series(x))
         .T)
pvals_against_nb = pvals_against_nb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

   gene_dropout_gene_dropout_logits_all gene_dropout_gene_dropout_logits_test  \
0                                   NaN                                   NaN   
1                                   NaN                                   NaN   
2                                   NaN                                   NaN   
3                                   NaN                                   NaN   
4                                   NaN                                   NaN   
5                                   NaN                                   NaN   
6                                   NaN                                   NaN   
7                                   NaN                                   NaN   
8                                   NaN                                   NaN   
9                                   NaN                                   NaN   
10                                  NaN                                   NaN   
11                          

   gene_dropout_gene_dropout_logits_all gene_dropout_gene_dropout_logits_test  \
0                                   NaN                                   NaN   
1                                   NaN                                   NaN   
2                                   NaN                                   NaN   
3                                   NaN                                   NaN   
4                                   NaN                                   NaN   
5                                   NaN                                   NaN   
6                                   NaN                                   NaN   
7                                   NaN                                   NaN   
8                                   NaN                                   NaN   
9                                   NaN                                   NaN   
10                                  NaN                                   NaN   
11                          

                 gene_dropout_gene_dropout_logits_all  \
0   [-6.4891915, -6.6107287, -2.4511063, -3.765836...   
1   [-6.4097958, -6.54981, -2.4831388, -3.4823968,...   
2   [-6.9322295, -7.088309, -2.4849832, -3.8177607...   
3   [-6.8156176, -6.9300094, -2.481998, -3.8837304...   
4   [-6.533313, -6.704564, -3.2697275, -3.3706708,...   
5   [-6.4305825, -6.643031, -3.853872, -2.9018567,...   
6   [-6.8870325, -7.0106363, -2.710241, -4.0867844...   
7   [-6.3320646, -6.453791, -3.056844, -3.6053588,...   
8   [-6.766125, -6.955121, -2.3919704, -4.0626144,...   
9   [-6.9222574, -7.051472, -3.1190093, -4.023244,...   
10  [-6.8211775, -6.981003, -2.3851044, -4.1201253...   
11  [-7.007531, -7.1925683, -3.265687, -3.8130143,...   
12  [-6.5777493, -6.703954, -2.4780445, -3.7198782...   
13  [-6.6413035, -6.747149, -2.2410612, -3.5343199...   
14  [-6.8600683, -7.027371, -2.581381, -3.7135718,...   
15  [-6.2373843, -6.3446245, -2.5217135, -3.610991...   
16  [-6.476118, -6.6314816, -3.

   gene_dropout_gene_dropout_logits_all gene_dropout_gene_dropout_logits_test  \
0                                   NaN                                   NaN   
1                                   NaN                                   NaN   
2                                   NaN                                   NaN   
3                                   NaN                                   NaN   
4                                   NaN                                   NaN   
5                                   NaN                                   NaN   
6                                   NaN                                   NaN   
7                                   NaN                                   NaN   
8                                   NaN                                   NaN   
9                                   NaN                                   NaN   
10                                  NaN                                   NaN   
11                          

   gene_dropout_gene_dropout_logits_all gene_dropout_gene_dropout_logits_test  \
0                                   NaN                                   NaN   
1                                   NaN                                   NaN   
2                                   NaN                                   NaN   
3                                   NaN                                   NaN   
4                                   NaN                                   NaN   
5                                   NaN                                   NaN   
6                                   NaN                                   NaN   
7                                   NaN                                   NaN   
8                                   NaN                                   NaN   
9                                   NaN                                   NaN   
10                                  NaN                                   NaN   
11                          

                 gene_dropout_gene_dropout_logits_all  \
0   [-6.4891915, -6.6107287, -2.4511063, -3.765836...   
1   [-6.4097958, -6.54981, -2.4831388, -3.4823968,...   
2   [-6.9322295, -7.088309, -2.4849832, -3.8177607...   
3   [-6.8156176, -6.9300094, -2.481998, -3.8837304...   
4   [-6.533313, -6.704564, -3.2697275, -3.3706708,...   
5   [-6.4305825, -6.643031, -3.853872, -2.9018567,...   
6   [-6.8870325, -7.0106363, -2.710241, -4.0867844...   
7   [-6.3320646, -6.453791, -3.056844, -3.6053588,...   
8   [-6.766125, -6.955121, -2.3919704, -4.0626144,...   
9   [-6.9222574, -7.051472, -3.1190093, -4.023244,...   
10  [-6.8211775, -6.981003, -2.3851044, -4.1201253...   
11  [-7.007531, -7.1925683, -3.265687, -3.8130143,...   
12  [-6.5777493, -6.703954, -2.4780445, -3.7198782...   
13  [-6.6413035, -6.747149, -2.2410612, -3.5343199...   
14  [-6.8600683, -7.027371, -2.581381, -3.7135718,...   
15  [-6.2373843, -6.3446245, -2.5217135, -3.610991...   
16  [-6.476118, -6.6314816, -3.

In [242]:
a = df.loc[df.model=='zifa_full', 'imputation_median_imputation_score']
b = df.loc[df.model=='zinb', 'imputation_median_imputation_score']
print(a.mean(), b.mean())
ttest_ind(a, b, alternative='smaller')

nan 5.085290393829346


C:\Users\ocliv\Anaconda3\envs\scvienv\lib\site-packages\statsmodels\stats\weightstats.py:132: RuntimeWarning: invalid value encountered in double_scalars
  return self.sum / self.sum_weights
C:\Users\ocliv\Anaconda3\envs\scvienv\lib\site-packages\statsmodels\stats\weightstats.py:837: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(var_pooled * (1. / d1.nobs + 1. /d2.nobs))
C:\Users\ocliv\Anaconda3\envs\scvienv\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\ocliv\Anaconda3\envs\scvienv\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\ocliv\Anaconda3\envs\scvienv\lib\site-packages\scipy\stats\_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


(nan, nan, 48.0)

In [243]:
pvals_against_zinb

model,nb,zinb
ll_ll,0.833279,0.5
imputation_median_imputation_score,0.009805,0.5
t_dropout_ks_stat,0.001744,0.5
t_ratio_ks_stat,0.335614,0.5
t_cv_ks_stat,0.990587,0.5


In [244]:
def get_summary(gby):
    res = {}
    res['mean'] = gby.mean()
    res['std'] = gby.std()
    res['pvals_against_nb'] = pvals_against_nb[gby.name]
    res['pvals_against_zi'] = pvals_against_zinb[gby.name]
    return pd.DataFrame(res).T

# df_summary = df.groupby('model')['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat',
#                                  't_ratio_ks_stat', 't_cv_ks_stat'].agg(['mean', 'std']).T
df_summary = df.groupby('model')[metrics].apply(get_summary)
df_summary


def my_styler(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
#     print(val)
    style = ['background-color: yellow' if v < 0.05 else '' for v in val]
    return style

df_summary = df_summary.stack().unstack(1).sort_index(level=1).swaplevel()
style = df_summary.style.apply(my_styler, subset=['pvals_against_nb', 'pvals_against_zi'], axis=1)
print(dataset_name)
style
# df_summary.loc[pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :]].style.background_gradient()
# df_summary.style.apply(my_styler, subset=pd.IndexSlice[:, 'pvals'], axis=0)
# df_summary = df_summary.style.apply(my_styler, subset=pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :], axis=1).render()
# df_summary.pivot(index=['model', ['mean', 'std', 'pvals']], columns=metrics)
# style


log_poisson_nb_dataset_8000


In [245]:
dropouts = df.loc[df.model == 'zinb'].gene_dropout_gene_dropout_probs_all
dropouts[0].mean()

0.009986183

In [214]:
# save as png
import imgkit
imgkit.from_string(style, os.path.join(data_path, '{}.png'.format(dataset_name)))

ModuleNotFoundError: No module named 'imgkit'

In [ ]:
# save as latex table

In [ ]:
text_file = open(os.path.join(data_path, 'latex_table.txt'), "w")
text_file.write(df_summary.to_latex())
text_file.close()